In [55]:
!pip install bert-for-tf2
!pip install sentencepiece

You should consider upgrading via the '/home/nobu_yamaguchi/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/nobu_yamaguchi/anaconda3/bin/python -m pip install --upgrade pip' command.


In [3]:
!pip3 install bert-for-tf2

You should consider upgrading via the '/home/nobu_yamaguchi/anaconda3/bin/python -m pip install --upgrade pip' command.


In [27]:
!pip install tensorflow_hub

You should consider upgrading via the '/home/nobu_yamaguchi/anaconda3/bin/python -m pip install --upgrade pip' command.


In [1]:
import tensorflow as tf

import tensorflow_hub as hub
import pandas as pd
from tensorflow.keras import layers
import bert

In [2]:
import numpy as np
import pickle as cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

Using TensorFlow backend.


In [3]:
df_merge_quality = pd.read_csv('US_patent_abstract_5000_2015_with_title_1.csv')
df_merge_quality.shape

(5000, 30)

In [4]:
df = df_merge_quality[['claims_text', 'quality_rank']]

In [5]:
df.quality_rank.value_counts()

0    2896
1    2104
Name: quality_rank, dtype: int64

In [6]:
2896/5000

0.5792

In [7]:
df.shape

(5000, 2)

In [8]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    #sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    #sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [9]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [10]:
claims = []
sentences = list(df['claims_text'])
for sen in sentences:
    claims.append(preprocess_text(str(sen)))

In [11]:
print(df.columns.values)

['claims_text' 'quality_rank']


In [12]:
df.quality_rank.unique()

array([0, 1])

In [13]:
y = df.quality_rank

In [14]:
y[10]

1

In [15]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [72]:
def tokenize_claims(text_claims):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_claims))

In [17]:
from nltk import tokenize

In [58]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model


In [25]:
max_sent = 30
max_sent_length = 30
sent_to_token = []
tokenized_claims = np.zeros(shape=(5000, max_sent, max_sent_length), dtype='int32')
#tokenized_claims = np.zeros((len(claims), max_sent, max_sent_length), dtype='int32')

#for i, sentences in enumerate(claims):
#    tokenized_claim.append("[CLS]")
#    for j, sent in enumerate(sentences):
#        if j < MAX_SENTS:
#            wordTokens = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))
#            k = 0
#            for _, word in enumerate(wordTokens):
#            #for word in wordTokens:
#                if word in tokenizer.word_index.keys():
#                    if (k < MAX_SENT_LENGTH) and (tokenizer.word_index[word] < MAX_NB_WORDS):
#                        tokenized_claims[i, j, k] = tokenizer.word_index[word]
#                        k = k + 1

for i, claim in enumerate(claims):
    sent_to_token = []
    tokenized_claim = []
#    tokenized_claim[i] = ["[CLS]"]
    # divide claims into multiple sentences
    sent_list = re.split(r'[.!?]+', claim)[:max_sent]
    
    k = 0
    for j, sent in enumerate(sent_list[:max_sent]): # for each sentence 
        temp = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))[:max_sent_length]
        if len(temp) < max_sent_length: # if temp length is less than max_sent_length, add 0
            for cal in range(max_sent_length-len(temp)):
                temp.append(0)
        sent_to_token.append(temp)
        k += 1
#        if len(sent_to_token) < max_sent_length - 1:
#            for j in range(max_sent_length-len(sent_to_token)-1):
#                sent_to_token.append(0 for m in range(max_sent_length-len(sent_to_token)-j))
    if k < 30:
        for p in range(30 - len(sent_list)):
            sent_to_token.append([0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
    #tokenized_claim.append([i, k, sent_to_token])
#    k += 1
#    if i == max_sent: 
#        break 
#    if k < max_sent:
#        while k <= max_sent:
#            tokenized_claim.append([0 for p in range(30)])
#            k += 1
#    tokenized_claim.append("[SEP]")
    tokenized_claims[i] = np.array(sent_to_token)
    
    

In [26]:
tokenized_claims[0]

array([[ 1015,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [ 1037,  5080,  9605,  1024,  3638,  2000,  3573,  8128,  1010,
         2073,  1996,  5080,  2003,  1037,  2034,  5080,  1025,  1998,
         1037, 13151,  2000, 15389,  1996,  8128,  2000,  1024,  4374,
         1037,  2034,  4471],
       [ 1016,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0],
       [ 1996,  5080,  1997,  4366,  1015,  1010,  2073,  1996, 13151,
         1010,  2043,  6364,  1996,  2034,  2724,  2951,  1010,  2003,
         2582,  2000,  1024,  6709,  1010,  2241,  2006,  1996,  2034,
         2724,  2951,  1010],
       [ 1017,     0,     0,

In [73]:
tokenized_claims = [tokenize_claims(claim) for claim in claims]

Prerparing Data For Training

In [77]:
claims_with_len = [[claim, y[i], len(claim)]
                 for i, claim in enumerate(tokenized_claims)]

In [22]:
#np.random.shuffle(claims_with_len)

In [28]:
#claims_with_len.sort(key=lambda x: x[2])

In [75]:
sorted_claims_labels = [(claim_lab[0], claim_lab[1]) for claim_lab in claims_with_len]

In [76]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_claims_labels, output_types=(tf.int32, tf.int32))

In [78]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [79]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 9746), dtype=int32, numpy=
 array([[ 1015,  1012,  1037, ...,     0,     0,     0],
        [ 1996, 11028,  3555, ...,     0,     0,     0],
        [ 2054,  2003,  3555, ...,     0,     0,     0],
        ...,
        [ 1015,  1012,  2019, ...,     0,     0,     0],
        [ 1996, 11028,  3555, ...,     0,     0,     0],
        [ 2054,  2003,  3555, ...,     0,     0,     0]], dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
        1, 0, 0, 0, 1, 1, 0, 0, 1, 1], dtype=int32)>)

In [80]:
TOTAL_BATCHES = np.math.ceil(len(sorted_claims_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 20
#batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [54]:
type(y)

pandas.core.series.Series

In [58]:
data = np.zeros((5000, 30, 30), dtype='int32')

In [27]:
x_train = np.array(tokenized_claims)[:4000]
x_test = np.array(tokenized_claims)[4000:]
y_train = np.array(y)[:4000]
y_test = np.array(y)[4000:]

In [24]:
a = 0
for element in test_data:
    a += len(element[1])
a

224

In [25]:
i = 0
for element in test_data:
    i += sum(element[1])
i

<tf.Tensor: shape=(), dtype=int32, numpy=100>

In [40]:
1-48/224

0.7857142857142857

In [22]:
from keras.engine.topology import Layer, InputSpec
from tensorflow.python.keras.layers import Lambda

In [28]:
class AttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(AttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        print(len(input_shape))
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)), name='W')
        self.b = K.variable(self.init((self.attention_dim, )), name='b')
        self.u = K.variable(self.init((self.attention_dim, 1)), name='u')
        self.trainable_weights = [self.W, self.b, self.u]
        super(AttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
 
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b)) #

        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)


        #if mask is not None:
        #    # Cast the mask to floatX to avoid float64 upcasting in theano
        #    print('mask')
        #    ait *= K.cast(mask, K.floatx())

        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        ait = K.expand_dims(ait)
 
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=200,
                 MAX_SENT_LENGTH=30,
                 MAX_SENTS=30,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.4,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.lstm = Bidirectional(GRU(100, return_sequences=True))
        
        self.l_att = AttLayer(100)
        self.sentEncoder = Model(Input(shape=(MAX_SENT_LENGTH,), dtype='int32'), )
        
        
        self.claims_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
        self.claims_encoder = TimeDistributed(self.sentEncoder)
        
        self.l_lstm_sent = Bidirectional(GRU(30, return_sequences=True))
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dense_2 = layers.Dense(units=dnn_units, activation="relu")
        self.dense_3 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.lstm(l) 
        l_2 = self.l_att(l_1)
        l_3 = self.sentEncoder
        
        l_4 = self.claims_input
        
        l_5 = claims_encoder(l_4)
        l_6 = l_lstm_sent(l_5)
        
        concatenated = self.dense_1(concatenated)
        concatenated = self.dense_2(concatenated)
        concatenated = self.dense_3(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [29]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.4

NB_EPOCHS = 5

text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        MAX_SENT_LENGTH=30,
                        MAX_SENTS=30,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [30]:
sentence_input = Input(shape=(30,), dtype='int32')
#sentence_input = Input(shape=(None,), dtype='int32')
#embedded_sequences = Embedding(VOCAB_LENGTH)(sentence_input)
embedded_sequences = Embedding(VOCAB_LENGTH, EMB_DIM)(sentence_input)
l_lstm = Bidirectional(GRU(100, return_sequences=True))(embedded_sequences)
l_att = AttLayer(100)(l_lstm)
sentEncoder = Model(sentence_input, l_att)

3
tracking <tf.Variable 'att_layer_1/W:0' shape=(200, 100) dtype=float32> W
tracking <tf.Variable 'att_layer_1/b:0' shape=(100,) dtype=float32> b
tracking <tf.Variable 'att_layer_1/u:0' shape=(100, 1) dtype=float32> u


In [32]:
claims_input = Input(shape=(30, 30), dtype='int32')
claims_encoder = TimeDistributed(sentEncoder)(claims_input)
l_lstm_sent = Bidirectional(GRU(30, return_sequences=True))(claims_encoder)

In [33]:
l_att_sent = AttLayer(100)(l_lstm_sent)

3
tracking <tf.Variable 'att_layer_2/W:0' shape=(60, 100) dtype=float32> W
tracking <tf.Variable 'att_layer_2/b:0' shape=(100,) dtype=float32> b
tracking <tf.Variable 'att_layer_2/u:0' shape=(100, 1) dtype=float32> u


In [34]:
#l_att_dense = Dense(100, activation='relu')(l_att_sent)
l_att_dense = Dense(100, activation='relu')(l_lstm_sent)
l_att_dense = Dense(100, activation='relu')(l_att_dense)
l_att_dense = Dense(100, activation='relu')(l_att_dense)
l_att_sent_drop = Dropout(rate=0.2)(l_att_dense)
preds = Dense(2, activation='sigmoid')(l_att_sent_drop)
model = Model(claims_input, preds)

In [35]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 30, 30)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 30, 200)           6305200   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 30, 60)            41580     
_________________________________________________________________
dense_1 (Dense)              (None, 30, 100)           6100      
_________________________________________________________________
dense_2 (Dense)              (None, 30, 100)           10100     
_________________________________________________________________
dense_3 (Dense)              (None, 30, 100)           10100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 100)           0   

In [36]:
model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["acc"])

In [110]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["acc"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

NameError: name 'text_model' is not defined

In [37]:
model.fit(x_train, y_train, validation_data=(x_test, y_test),
          epochs=8, batch_size=50)

ValueError: Error when checking target: expected dense_4 to have 3 dimensions, but got array with shape (4000, 1)

In [119]:
model.fit(train_data, epochs=NB_EPOCHS, validation_data=test_data)

AttributeError: 'SkipDataset' object has no attribute 'ndim'